# ch 19_2 huggingface tokenizers

이번 챕터에서는 huggingface에서 제공하는 tokenizers 라이브러리를 이용해서 직접 corpus 데이터 셋을 가지고 subword tokenizer를 학습시켜 보겠습니다.

## 데이터 셋 준비

### 네이버 영화 리뷰 데이터 셋

네이버 영화 리뷰 데이터 셋으로 간단히 학습을 진행해보겠습니다.

In [215]:
import pandas as pd

In [216]:
train_df = pd.read_csv("./data/naver_reviews_train.csv")

In [217]:
train_df

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


### 데이터 셋 전처리

결측치를 제거하고, 특수문자나 한자를 제거해주겠습니다.

In [218]:
train_df.isnull().sum()

document    5
label       0
dtype: int64

In [219]:
train_df = train_df.dropna()

In [249]:
import re

# 특수 문자 제거 및 소문자화
def preprocess_text(text: str) -> str:
    text = text.lower()
    # text = re.sub(r"[^ㄱ-힣a-zA-Z0-9!?.,\']", r" ", text)
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z!?.,\' ]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [251]:
example = "亞 최고의 영화...!! 단연코 BEST 보는 내내 눈물 ㅠㅠ"
preprocess_text(example)

'최고의 영화...!! 단연코 best 보는 내내 눈물 ㅠㅠ'

In [252]:
train_df["document"] = train_df["document"].apply(lambda x: preprocess_text(x))

/var/folders/wx/2vw1xwz972zdxff5h8gzrml00000gn/T/ipykernel_13815/1102241259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["document"] = train_df["document"].apply(lambda x: preprocess_text(x))


In [253]:
train_df

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## huggingface tokenizers

huggingface는 AI 스타트업으로 오픈 소스 라이브러리로 유명합니다. 특히 NLP 분야에서는 huggingface에서 제공하는 트랜스포머 모델을 사용하는 것이 거의 표준으로 자리잡았습니다. 주요 라이브러리는 아래와 같습니다. 

- transformers: 트랜스포머 기본 모델과 이를 응용한 NLP 모델들을 제공
- tokenizers: subword tokenizer 제공

huggingface의 tokenizers는 subword tokenizer의 구현체입니다. 이를 사용하여 tokenizer를 학습시켜 보겠습니다.

In [255]:
!pip install tokenizers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### Trainer
WordPiece 기반의 subword tokenizer를 만들어보겠습니다.  먼저 tokenizer를 학습시키기 위한 trainer를 만들어줍니다. 이 때, special tokens를 넣어주어야 하는데, 각 토큰의 의미는 다음과 같습니다.

- [PAD]: padding의 약자로 문장 간에 길이를 맞춰주기 위해 일부러 채워넣은 토큰을 의미합니다.
- [UNK]: unknown의 약자로 인식하지 못한 토큰을 나타냅니다.
- [SOS]: start of sentence의 약자로 문장의 시작을 표시해줍니다.
- [EOS]: end of sentence의 약자로 문장의 끝을 표시해줍니다.

In [260]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=10000,
    min_frequency=5,
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"]
)

### Tokenizer

이제 tokenizer 객체를 만들어주고 trainer를 이용해서 학습시켜 줍니다.

In [266]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece 
from tokenizers.pre_tokenizers import Whitespace

def batch_iterator(df, batch_size=1000):
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        yield batch_df["document"] 

In [267]:
tokenizer = Tokenizer(WordPiece())
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(batch_iterator(train_df),  trainer=trainer)

## tokenizer 확인

tokenizer가 잘 학습되었는지 확인해보겠습니다.

### vocab 확인

먼저 tokenizer vocab에 어떤 토큰들이 추가되었는지 살펴보겠습니다.

In [277]:
vocabs = tokenizer.get_vocab()
sorted_vocabs = sorted(vocabs.items(), key=lambda x: x[1])

In [ ]:
print(sorted_vocabs)

4000 ~ 5000개까지는 초기 기본 토큰들로 채워져 있고, 그 뒤로는 함께 자주 등장하는 글자끼리 묶인 토큰들을 확인할 수 있습니다. 앞에 ##이 붙은 토큰들은 단어의 시작 지점이 아닌 위치에 등장하는 토큰들입니다.

### 샘플 토큰화

예시 문장들을 토큰화 해보겠습니다.

In [284]:
samples = [
    "너무 재밌어요, 꿀잼 인정!",
    "보다가 중간에 졸았습니다 ㅠㅠ 비추에요"
]

In [285]:
for sample in samples:
    output = tokenizer.encode(sample)
    print(output.ids)
    print(output.tokens)

[4876, 5383, 6, 6014, 6295, 4]
['너무', '재밌어요', ',', '꿀잼', '인정', '!']
[5278, 5824, 1843, 6945, 5033, 6190, 5372]
['보다가', '중간에', '졸', '##았습니다', 'ㅠㅠ', '비추', '##에요']


### 저장

잘 학습된 것을 확인했다면 파일에 저장하겠습니다.

In [286]:
tokenizer.save("./data/tokenizer.json")

## 정리

이번 챕터에서는 huggingface에서 제공하는 tokenizers 라이브러리를 이용해서 직접 subword tokenizer를 만들어보았습니다. subword tokenizer는 활용도가 매우 높으니, 사용법을 잘 기억해주시기 바랍니다.